In [131]:
import pandas as pd
from datetime import datetime, timedelta

start_year = pd.Timestamp('2025-01-01', tz='UTC')


In [132]:
import json
import pandas as pd
from glob import glob
import os

# Load dataFrame
folder_path = "./Documents/spotifyData"
files = glob(os.path.join(folder_path, "*.json"))

dfs = []
for file in files:
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
        dfs.append(pd.DataFrame(data))
df = pd.concat(dfs, ignore_index=True)

In [133]:
# Filter df to include only entries from 2025.
df["ts"] = pd.to_datetime(df["ts"])
df = df[df["ts"] >= start_year]

In [134]:
# Rename Columns
df = df.rename(columns={
    "ts": "timestamp",
    "master_metadata_track_name": "track_name",
    "master_metadata_album_artist_name": "artist_name",
    "master_metadata_album_album_name": "album_name",
})

# Remove missing data/non-songs
df.dropna(subset=['track_name', 'artist_name'])

,timestamp,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,spotify_track_uri,episode_name,...,audiobook_uri,audiobook_chapter_uri,audiobook_chapter_title,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
9089,2025-01-01 02:42:23+00:00,ios,117,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,None,None,None,clickrow,endplay,False,True,False,1735699343,False
9090,2025-01-01 02:42:23+00:00,ios,15994,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,None,None,None,trackdone,endplay,False,True,False,1735687171,False
9091,2025-01-01 02:42:36+00:00,ios,12864,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,None,None,None,clickrow,endplay,False,True,False,1735699343,False
9092,2025-01-01 02:45:49+00:00,ios,193213,US,174.247.179.81,Single Ladies (Put a Ring on It),Beyoncé,I AM...SASHA FIERCE,spotify:track:2ZBNclC5wm4GtiWaeh0DMx,None,...,None,None,None,playbtn,trackdone,True,False,False,1735699356,False
9093,2025-01-01 02:49:32+00:00,ios,222733,US,174.247.179.81,Forget You,CeeLo Green,The Lady Killer,spotify:track:7AqISujIaWcY3h5zrOqt5v,None,...,None,None,None,trackdone,trackdone,True,False,False,1735699549,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30864,2025-12-18 23:29:16+00:00,windows,141013,US,75.164.31.180,Old Friends,Pinegrove,Cardinal,spotify:track:4ykQ18swXrkrrbwsZKxQo7,None,...,None,None,None,clickrow,endplay,False,True,False,1766100415,False
30865,2025-12-18 23:31:21+00:00,windows,110993,US,75.164.31.180,Alaska,Pinegrove,11:11,spotify:track:6PsqixVNSE6N4WamoMFISK,None,...,None,None,None,clickrow,endplay,False,True,False,1766100556,False
30866,2025-12-18 23:31:37+00:00,windows,14993,US,75.164.31.180,Need 2,Pinegrove,Everything So Far,spotify:track:1BDRKVuooLvqayamtAEV4z,None,...,None,None,None,clickrow,endplay,False,True,False,1766100681,False
30867,2025-12-18 23:36:08+00:00,windows,248290,US,75.164.31.180,In the Aeroplane Over the Sea,Neutral Milk Hotel,In the Aeroplane Over the Sea,spotify:track:1NiAMp0CzUdG6e97VYRB1a,None,...,None,None,None,clickrow,trackdone,False,False,False,1766100697,False


In [135]:
# Add series' to df
df["minutes_played"] = df["ms_played"] / 60000
df["month"] = df["timestamp"].dt.to_period("M").astype(str)
df["day_of_week"] = df["timestamp"].dt.day_name()
df["date"] = df["timestamp"].dt.date
df["hour"] = df["timestamp"].dt.hour

C:\Users\jiglo\AppData\Local\Temp\ipykernel_31368\460452254.py:3: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["timestamp"].dt.to_period("M").astype(str)


In [136]:
import sqlite3

conn = sqlite3.connect("spotify.db")
df.to_sql("spotify_streams", conn, if_exists="replace", index=False)

21780

In [137]:
# query the database
query = "SELECT * FROM spotify_streams LIMIT 5;"
pd.read_sql(query, conn)


,timestamp,platform,ms_played,conn_country,ip_addr,track_name,artist_name,album_name,spotify_track_uri,episode_name,...,shuffle,skipped,offline,offline_timestamp,incognito_mode,minutes_played,month,day_of_week,date,hour
0,2025-01-01 02:42:23+00:00,ios,117,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,0,1,0,1735699343,0,0.001950,2025-01,Wednesday,2025-01-01,2
1,2025-01-01 02:42:23+00:00,ios,15994,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,0,1,0,1735687171,0,0.266567,2025-01,Wednesday,2025-01-01,2
2,2025-01-01 02:42:36+00:00,ios,12864,US,174.247.179.81,What's Up?,Mom Jeans.,Sweet Tooth,spotify:track:141alNiSd9vG4Lb22BLHWM,None,...,0,1,0,1735699343,0,0.214400,2025-01,Wednesday,2025-01-01,2
3,2025-01-01 02:45:49+00:00,ios,193213,US,174.247.179.81,Single Ladies (Put a Ring on It),Beyoncé,I AM...SASHA FIERCE,spotify:track:2ZBNclC5wm4GtiWaeh0DMx,None,...,1,0,0,1735699356,0,3.220217,2025-01,Wednesday,2025-01-01,2
4,2025-01-01 02:49:32+00:00,ios,222733,US,174.247.179.81,Forget You,CeeLo Green,The Lady Killer,spotify:track:7AqISujIaWcY3h5zrOqt5v,None,...,1,0,0,1735699549,0,3.712217,2025-01,Wednesday,2025-01-01,2


In [138]:
# define views for db
cursor = conn.cursor()

views = {
    "artist_summary" : """ -- Metrics by artist
    CREATE VIEW artist_summary AS
    SELECT
        artist_name,
        SUM(minutes_played) AS total_minutes,
        COUNT(*) AS total_plays,
        COUNT(DISTINCT track_name) AS unique_tracks
        FROM spotify_streams
        GROUP BY artist_name;
    """,
    "daily_listening" : """ -- Listening minutes by day
    CREATE VIEW daily_listening AS
    SELECT 
        date,
        SUM(minutes_played) AS daily_minutes
    FROM spotify_streams
    GROUP BY date;
    """,
    "listening_by_day" : """ -- Listening minutes by day of the week
    CREATE VIEW listening_by_day AS
    SELECT 
        day_of_week,
        SUM(minutes_played) AS day_of_week_minutes
    FROM spotify_streams
    GROUP BY day_of_week;
    """,
    "hourly_listening" : """ -- Listening minutes by hour
    CREATE VIEW hourly_listening AS
    SELECT
        hour,
        SUM(minutes_played) AS hourly_minutes
    FROM spotify_streams
    GROUP BY hour;
    """
    }
        

In [139]:
# execute views
for view_name, view_sql in views.items():
    cursor.execute(f"DROP VIEW IF EXISTS {view_name};")
    cursor.execute(view_sql)


In [140]:
# commit changes and verify that views were created
conn.commit()
check_views = """
    SELECT name FROM sqlite_master
    WHERE type = 'view';
    """
pd.read_sql(check_views, conn)

,name
0,artist_summary
1,daily_listening
2,listening_by_day
3,hourly_listening


In [141]:
conn.close()